In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.layers import Dense, Concatenate,Dropout, Input, Embedding, LSTM, Bidirectional,Flatten,SpatialDropout1D, CuDNNLSTM ,Conv1D,GlobalMaxPooling1D
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,SpatialDropout1D
from sklearn.preprocessing import LabelBinarizer
import keras
import keras.preprocessing.text
from keras.preprocessing import sequence
from random import *
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,SpatialDropout1D, MaxPooling1D
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import copy
from sklearn.metrics import classification_report
from keras.models import load_model
import keras.backend.tensorflow_backend as K
import tensorflow as tf
import pickle

Using TensorFlow backend.


In [2]:
pos_tagger1 = Mecab()
pos_tagger2 = Twitter()

pos_tagger = pos_tagger1

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [3]:
#test_df = pd.read_excel("./data/testset.xlsx",header=None,sheet_name="쓰레기폐기물")
#thres = 0.6
test_df = pd.read_csv("/media/Data/Naver_Comments/result/naver_comments_long.csv")
fname = "네이버_결과.xlsx"

In [4]:
test_df.columns=['no','data','title','comments']

In [5]:
X_test = test_df['comments'].tolist()

In [6]:
BASE_CODE, CHOSUNG, JUNGSUNG = 44032, 588, 28

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


def convert(test_keyword):
    split_keyword_list = list(test_keyword)
    #print(split_keyword_list)

    result = list()
    for keyword in split_keyword_list:
        # 한글 여부 check 후 분리
       # print(keyword)
        if re.match('.*[ㄱ-ㅎㅏ-ㅣ가-힣]+.*', keyword) is not None:
            try:
                char_code = ord(keyword) - BASE_CODE
                char1 = int(char_code / CHOSUNG)
                if char1==0:  
                    result.append('')
                result.append(CHOSUNG_LIST[char1])
               # print('초성 : {}'.format(CHOSUNG_LIST[char1]))
                char2 = int((char_code - (CHOSUNG * char1)) / JUNGSUNG)
                if char2==0:
                    result.append('')
                result.append(JUNGSUNG_LIST[char2])
              #  print('중성 : {}'.format(JUNGSUNG_LIST[char2]))
                char3 = int((char_code - (CHOSUNG * char1) - (JUNGSUNG * char2)))
                if char3==0:
                    result.append('')
                else:
                    result.append(JONGSUNG_LIST[char3])
               # print('종성 : {}'.format(JONGSUNG_LIST[char3]))
            except:
                result.append(keyword)
        else:
            result.append(keyword)
    # result
    return "".join(result)

In [ ]:
# Mecab 형태소 분석기 사용
t_result_list = []
# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
        
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        
        pos = list(pd.DataFrame(pos_tagger.pos(t))[1])
        t = np.array(pos_tagger.morphs(t))
        t = t.tolist()
        t_result_list.append(" ".join(t))
    except:
        t_result_list.append("중립")
        
X_test1 = t_result_list
print("완료1")
t_result_list = []
for i,s in enumerate(X_test):
    s = str(s)
    s = s.strip()
    temp_list = s.split(" ")
    
   # 특수문자 제거  
    s = re.sub('#', '', s)
    s = re.sub('\'', '', s)
    s = re.sub('\「', '', s)
    s = re.sub('\」', '', s)
    s = re.sub('\,', '', s)
    s = re.sub('#', '', s)
    s = re.sub('@', '', s)
    s = re.sub('\+', '', s)
    s = re.sub('[A-z]', '', s)
    s = re.sub('[0-9]', '', s)
    s = s.strip()
    
    t_result_list.append(s)

X_all_list2 = t_result_list

for i, comment in enumerate(X_all_list2):
    temp = convert(comment)
    
    # 빈칸제거
    temp = temp.replace(" ","")
    temp2 = ""
    for t in temp:
        temp2 = temp2 + t +  " "
    temp2 = temp2[0:-1]
    X_all_list2[i] = temp2

X_test2 = X_all_list2
print("완료2")
# Mecab 형태소 분석기 사용
t_result_list = []

# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        t = t.strip()
        
        # 조사 제거
        t = t.replace(" ", "") 
        t = list(t)
        t_result_list.append(" ".join(t))
        
    except:
        t_result_list.append("중립")

X_test3 = t_result_list

# Mecab 형태소 분석기 사용
t_result_list = []

# 형태소 분석결과 아무것도 안나오는 경우
for idx,t in enumerate(X_test):
    try:
        t = re.sub('#', '', t)
        t = re.sub('\'', '', t)
        t = re.sub('\「', '', t)
        t = re.sub('\」', '', t)
        t = re.sub('\,', '', t)
        t = re.sub('#', '', t)
        t = re.sub('@', '', t)
        t = re.sub('\+', '', t)
        t = re.sub('[A-z]', '', t)
        t = re.sub('[0-9]', '', t)
        
        # 조사 제거
        pos = list(pd.DataFrame(pos_tagger2.pos(t))[1])
        t = np.array(pos_tagger2.morphs(t))
        t = t.tolist()
        t_result_list.append(" ".join(t))        
    except:
        print(t)
        t_result_list.append("중립")

X_test4 = t_result_list

완료1
완료2


In [ ]:
max_slen1 = 200
max_slen2 = 500
max_slen3 = 200
max_slen4 = 200

In [18]:
# saving
with open('./model/tk1_1.pickle', 'rb') as handle:
    tk1 = pickle.load(handle)
with open('./model/tk2_1.pickle', 'rb') as handle:
    tk2 = pickle.load(handle)
with open('./model/tk3_1.pickle', 'rb') as handle:
    tk3 = pickle.load(handle)
with open('./model/tk4_1.pickle', 'rb') as handle:
    tk4 = pickle.load(handle)

### 모델 로드

In [19]:
X_test1 = tk1.texts_to_sequences(X_test1)
X_test1 = sequence.pad_sequences(X_test1, maxlen=max_slen1,padding='pre')
X_test2 = tk2.texts_to_sequences(X_test2)
X_test2 = sequence.pad_sequences(X_test2, maxlen=max_slen2,padding='pre')
X_test3 = tk3.texts_to_sequences(X_test3)
X_test3 = sequence.pad_sequences(X_test3, maxlen=max_slen3,padding='pre')
X_test4 = tk4.texts_to_sequences(X_test4)
X_test4 = sequence.pad_sequences(X_test4, maxlen=max_slen4,padding='pre')

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [23]:
from keras.models import load_model
with tf.device('/cpu:0'):
    model = load_model('./model/senti_model_1.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [24]:
sub_X_test1 = X_test1[1:1000000]
sub_X_test2 = X_test2[1:1000000]
sub_X_test3 = X_test3[1:1000000]
sub_X_test4 = X_test4[1:1000000]

In [25]:
with tf.device('/cpu:0'):
    test_result = model.predict([sub_X_test1,sub_X_test2,sub_X_test3,sub_X_test4])

In [26]:
test_result = pd.DataFrame(test_result)

In [27]:
test_result.shape

(999999, 2)

In [28]:
test_result.to_csv("naver_result.csv")